In [ ]:
#Data Loader
import io
import pandas as pd
import requests
import numpy as np

if 'data_loader' not in globals():
    from mage_ai.data_preparation.decorators import data_loader
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test


@data_loader
def load_data_from_api(*args, **kwargs):
    """
    Template for loading data from API
    """
    url = 'https://storage.googleapis.com/mootod_data_project/cleaned_datatable.csv'
    response = requests.get(url)
    return pd.read_csv(io.StringIO(response.text), sep=',')


@test
def test_missing_values(df) -> None:
    """
    Template code for testing the presence of missing values and handling string values in almost numeric columns.
    """
    missing_values = df.isnull().sum().sum()
    if missing_values > 0:
        print(f"The data contains {missing_values} missing value(s).")


        df = df.fillna(method='ffill')
        numeric_columns = df.select_dtypes(include=[np.number]).columns
        df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median())
    #just only fill for load
    assert df.isnull().sum().sum() == 0, 'The data contains missing values.'




In [ ]:
#Transformer
import pandas as pd
import numpy as np
if 'transformer' not in globals():
    from mage_ai.data_preparation.decorators import transformer
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test


@transformer
def transform(df, *args, **kwargs):
    """
    Template code for a transformer block.

    Add more parameters to this function if this block has multiple parent blocks.
    There should be one parameter for each output variable from each parent block.

    Args:
        data: The output from the upstream parent block
        args: The output from any additional upstream blocks (if applicable)

    Returns:
        Anything (e.g. data frame, dictionary, array, int, str, etc.)
    """
    # Specify your transformation logic here
    df=df.drop_duplicates().reset_index(drop=True)
    df['order_id']=df.index
    #handle missing value
    df=df.fillna(method='ffill')

    ###############################################################

    df['time'] = pd.to_datetime(df['time'], format='%d/%m/%Y %H:%M')
    datetime_dim = df[['time']].reset_index(drop=True)
    datetime_dim['sale_time']=datetime_dim['time'].dt.time
    datetime_dim['sale_day']=datetime_dim['time'].dt.day
    datetime_dim['sale_month']=datetime_dim['time'].dt.month
    datetime_dim['sale_year']=datetime_dim['time'].dt.year

    datetime_dim['datetime_id']=datetime_dim.index
    datetime_dim = datetime_dim[['datetime_id','time','sale_time','sale_day','sale_month','sale_year']]
    ###############################################################
    vendor_name={
        1:"Noon",
        2:"Minnie"
    }
    vendor_dim=df[['vendor']].reset_index(drop=True)
    vendor_dim['vendor_id']=vendor_dim.index
    vendor_dim['vendor_name']=vendor_dim['vendor'].map(vendor_name)
    vendor_dim=vendor_dim[['vendor_id','vendor','vendor_name']]
    ###############################################################
    bill_type_name={
        1:"For_here",
        2:"To_go"
    }
    bill_type_dim=df[['bill_type']].reset_index(drop=True)
    bill_type_dim['bill_type_id']=bill_type_dim.index
    bill_type_dim['bill_type_name']=bill_type_dim['bill_type'].map(bill_type_name)
    bill_type_dim=bill_type_dim[['bill_type_id','bill_type','bill_type_name']]
    ###############################################################
    pickup_type_name={
        1:"served",
        2:"pickup_by_themselves",
        3:"LineMan",
        4:"GrabFood",
        5:"FoodPanda",
        6:"ShopeeFood",
        7:"Robinhood"
    }
    pickup_type_dim=df[['pickup_type']].reset_index(drop=True)
    pickup_type_dim['pickup_type_id']=pickup_type_dim.index
    pickup_type_dim['pickup_type_name']=pickup_type_dim['pickup_type'].map(pickup_type_name)
    pickup_type_dim=pickup_type_dim[['pickup_type_id','pickup_type','pickup_type_name']]
    ###############################################################
    payment_method_name={
        1:"transfer",
        2:"cash",
        3:"other"
    }
    payment_method_dim=df[['payment_method']].reset_index(drop=True)
    payment_method_dim['payment_method_id']=payment_method_dim.index
    payment_method_dim['payment_method_name']=payment_method_dim['payment_method'].map(payment_method_name)
    payment_method_dim=payment_method_dim[['payment_method_id','payment_method','payment_method_name']]
    ###############################################################
    fact_table = df.merge(datetime_dim, left_on='order_id', right_on='datetime_id') \
             .merge(vendor_dim, left_on='order_id', right_on='vendor_id') \
             .merge(bill_type_dim, left_on='order_id', right_on='bill_type_id') \
             .merge(pickup_type_dim, left_on='order_id', right_on='pickup_type_id') \
            .merge(payment_method_dim, left_on='order_id', right_on='payment_method_id')\
             [['order_id','ref_no', 'datetime_id', 'vendor_id',
               'bill_type_id', 'pickup_type_id', 'payment_method_id',
               'sale', 'payment', 'tip', 'change',]]
    ###############################################################
    return {"datetime_dim":datetime_dim.to_dict(orient='dict'),
    "vendor_dim":vendor_dim.to_dict(orient='dict'),
    "bill_type_dim":bill_type_dim.to_dict(orient='dict'),
    "pickup_type_dim":pickup_type_dim.to_dict(orient='dict'),
    "payment_method_dim":payment_method_dim.to_dict(orient='dict'),
    "fact_table":fact_table.to_dict(orient='dict')}


@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'


In [ ]:
#Data Exporter
from mage_ai.data_preparation.repo_manager import get_repo_path
from mage_ai.io.bigquery import BigQuery
from mage_ai.io.config import ConfigFileLoader
from pandas import DataFrame
from os import path

if 'data_exporter' not in globals():
    from mage_ai.data_preparation.decorators import data_exporter


@data_exporter
def export_data_to_big_query(data, **kwargs) -> None:
    """
    Template for exporting data to a BigQuery warehouse.
    Specify your configuration settings in 'io_config.yaml'.

    Docs: https://docs.mage.ai/design/data-loading#bigquery
    """
    config_path = path.join(get_repo_path(), 'io_config.yaml')
    config_profile = 'default'

    for key,value in data.items():
        table_id = 'projectmadt.mootod_preject_data.{}'.format(key)
        BigQuery.with_config(ConfigFileLoader(config_path, config_profile)).export(
            DataFrame(value),
            table_id,
            if_exists='replace',  # Specify resolution policy if table name already exists
        )
